In [1]:
import firebirdsql as kib
import pandas as pd
import copy as cp

In [2]:
#url de exportacion
url="../datasets_resultantes/interbase_2001/tablas_csv/"

#url de la base de datos
urlBD="F:/Proyecto_Demografico/datasets_fuentes/INTERBD_2001_06_VF.GDB"

#cargar los datos de acceso para la base de datos
dtAcceso = pd.read_csv('accesos.txt',sep=';' ,encoding='utf8')
usuario=dtAcceso[(dtAcceso["BASE"]=="INTERBASE_2001_06")].iloc[0,1]
clave=dtAcceso[(dtAcceso["BASE"]=="INTERBASE_2001_06")].iloc[0,2]


In [3]:
#conexion a la base de datos del 2000
conn = kib.connect(
    host='192.10.0.7',
    database=urlBD,
    port=3050,
    user=usuario,
    password=clave,
    charset='UNICODE_FSS'  
)

In [ ]:
# definicion del cursor y listar las tablas de la base de datos
#cursor para el manejo de las consultas
curr = conn.cursor()

# listar las tablas de la base de datos
curr.execute("SELECT RDB$RELATION_NAME FROM RDB$RELATIONS WHERE (RDB$SYSTEM_FLAG <> 1 OR RDB$SYSTEM_FLAG IS NULL) AND RDB$VIEW_BLR IS NULL ORDER BY RDB$RELATION_NAME;")

# resulset asignado a un dataframe
df_tablas=pd.DataFrame(curr.fetchall())
df_tablas.columns=["Tablas"]

print(df_tablas)


In [5]:
# esta funcion recibe un cursor de las consultas y devuelve el nombre de las columnas o campos
# los nombres de las columnas se obtienen de la posicion 0 del arreglo que devuelve "description" para cada campo
def columnas(cursor:any):
    return map( lambda x: x[0],cursor.description)


In [6]:
# retorna un dataframe de la tabla consultada a partir del parametro recibido
def consultaTabla(nombre_tabla:str , campos:str):
    #consultar tabla y cargarla en un dataframe
    try:
        # evaluar si el criterio de consulta es general o especifico
        if campos!="":
            curr.execute("SELECT "+campos+" FROM "+nombre_tabla+";")
        else:
            curr.execute("SELECT * FROM "+nombre_tabla+";")

        # se utiliza la funcion columnas para obtener la lista de columnas
        df_tabla=pd.DataFrame(data=curr.fetchall(),columns= columnas(curr))
    except kib.Error as error:
        print("error en consulta tabla",error)
    return df_tabla  

In [7]:
# exportar las tablas a un csv utilizando una funcion 
# recibe el nombre que recibira el archivo, el dataframe o cuerpo de la tabla y la ubicacion dende se creara el archivo
def exportaTabla(nombre_tabla:str,tabla:pd.DataFrame,url:str):
    tabla.to_csv(url+'tbl_'+nombre_tabla+'.csv', sep='|',lineterminator=";",quotechar='"',doublequote=True, index=False)

In [8]:
#ciclo de exportacion de la tablas
# consiste en llamar la funcion "exportarTabla" y enviarle como parametro el nombre la tabla, 
# el dataframe que retorna la funcion "consultaTabla" y la url a donde se exportara la tabla consultada
def cicloExportacion():
    for x in range(0,df_tablas.size,1):
        exportaTabla(df_tablas.iloc[x][0],consultaTabla(df_tablas.iloc[x][0],""),url) 

In [9]:
# completar direccion
def completarDir(region1:str,region2:str):
    temporal=""
        
    if len(region1)<2:
        temporal+="0"+region1
    else:
        if int(region1)>=21:
            region1=str(int(region1)-20)
        if len(region1)<2:
            region1="0"+region1
        temporal+=region1
    
    if len(region2)<2:
        temporal+="0"+region2
    else:
        temporal+=region2
    return temporal

In [1]:
#completar fecha
def completarF(fecha1:str,fecha2:str):
    temporal=""
    if len(fecha1)<4:
        temporal=fecha2[4:8]
    else:
        temporal=fecha1
    
    if(float(fecha1)>1982):
        temporal=fecha2[4:8]

    return temporal

In [11]:
#verificar direccion
def verificarDir(region1:str,region2:str):
    temporal=""
    if region1[0:2]=="19" or region1[2:4]=="80" or region1[2:4]=="90":
        temporal+=region2
    else:
        temporal+=region1
    return temporal


In [ ]:

#carga de los dataframes 
df_DEPARTAMENTOS=consultaTabla(df_tablas.iloc[3][0],"")
df_MUNICIPIOS=consultaTabla(df_tablas.iloc[6][0],"")
DF_cuerpo=consultaTabla(df_tablas.iloc[4][0]," identidad, nombre1,nombre2,apellido1,apellido2,sexo,anonac,mesnac,dianac,depdomi,munidomi")

#obtencion y completarcion de direcciones
df_datos=DF_cuerpo.copy()
df_datos[df_datos.columns[0]]=df_datos[df_datos.columns[0]].apply(lambda x: "0"+str(x) if len(str(x))<13 else str(x) ).astype(str)
df_datos["LUDOMI"]=list(map(completarDir,df_datos[df_datos.columns[9]].astype(str),df_datos[df_datos.columns[10]].astype(str)))
#df_datos["LUNAC"]=df_datos[df_datos.columns[0]].apply(lambda x: str(x)[0:4]).astype(str)
df_datos["LUNAC"]=df_datos[df_datos.columns[0]].apply(lambda x: completarDir(str(x)[0:2],str(x)[2:4])).astype(str)
df_datos["DEPNAC"]=df_datos[df_datos.columns[0]].apply(lambda x: str(x)[0:2]).astype(int)
df_datos["MUNINAC"]=df_datos[df_datos.columns[0]].apply(lambda x: str(x)[2:4]).astype(int)
df_MUNICIPIOS["LUDIR"]=list(map(completarDir,df_MUNICIPIOS[df_MUNICIPIOS.columns[0]].astype(str),df_MUNICIPIOS[df_MUNICIPIOS.columns[1]].astype(str)))

#limpieza, completacion y conversion de datos
df_datos[df_datos.columns[7]]=df_datos[df_datos.columns[7]].fillna(0)
df_datos[df_datos.columns[5]]=df_datos[df_datos.columns[5]].fillna(0)
df_datos[df_datos.columns[5]]=df_datos[df_datos.columns[5]].astype(float).astype(int)
df_datos[df_datos.columns[7]]=df_datos[df_datos.columns[7]].astype(int)
df_datos[df_datos.columns[8]]=df_datos[df_datos.columns[8]].fillna(0)
df_datos[df_datos.columns[8]]=df_datos[df_datos.columns[8]].astype(int)
df_datos[df_datos.columns[6]]=df_datos[df_datos.columns[6]].fillna(0)
df_datos[df_datos.columns[6]]=list(map(completarF,df_datos[df_datos.columns[6]].astype(str),df_datos[df_datos.columns[0]].astype(str)))
df_datos[df_datos.columns[6]]=df_datos[df_datos.columns[6]].astype(float).astype(int)
df_datos[df_datos.columns[12]]=list(map(verificarDir,df_datos[df_datos.columns[12]].astype(str),df_datos[df_datos.columns[11]].astype(str)))
df_datos[df_datos.columns[13]]=df_datos[df_datos.columns[13]].apply(lambda x: x-20 if x>20 else x)
df_datos[df_datos.columns[1]]=df_datos[df_datos.columns[1]].apply(lambda x: str(x).strip(".,0123456789-=`;@¦"))
df_datos[df_datos.columns[2]]=df_datos[df_datos.columns[2]].apply(lambda x: str(x).strip(".,0123456789-=`;@¦"))
df_datos[df_datos.columns[3]]=df_datos[df_datos.columns[3]].apply(lambda x: str(x).strip(".,0123456789-=`;@¦"))
df_datos[df_datos.columns[4]]=df_datos[df_datos.columns[4]].apply(lambda x: str(x).strip(".,0123456789-=`;@¦"))
#del df_datos[df_datos.columns[0]]
df_datos.info()
print(df_datos.head(3))

In [54]:
# exportar dataFrame, datos, departamentos, municipios
exportaTabla("DEPARTAMENTOS_VF",df_DEPARTAMENTOS,url)
exportaTabla("MUNICIPIOS_VF",df_MUNICIPIOS,url)
exportaTabla("DATOS_2001",df_datos,url)

In [12]:
#cerra las conexiones abiertas
conn.close()